In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

import tensorflow.keras as keras
import tensorflow  as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *

from tensorflow.keras.callbacks import *

from sklearn.metrics import mean_absolute_error
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

import os

In [28]:
frame = pd.read_csv(r'Predicting_Bike_Share_Demand_with_LSTM-master/hour.csv')
#frame = pd.read_csv(r'ausleihen_mit_wetter.csv')
df = pd.DataFrame(frame)
col = list(df.columns.values)

test = pd.DataFrame(frame)

df.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [32]:
df = df.drop(['casual','registered','instant'],axis=1)

In [33]:
df.head()

,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,16
1,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,40
2,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,32
3,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,13
4,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,1


In [52]:
df.dtypes

dteday        datetime64[ns]
season                 int64
yr                     int64
mnth                   int64
hr                     int64
holiday                int64
weekday                int64
workingday             int64
weathersit             int64
temp                 float64
atemp                float64
hum                  float64
windspeed            float64
cnt                    int64
dtype: object

In [56]:
#Starttime to date-format
df['dteday'] = pd.to_datetime(df['dteday'], errors='coerce')

In [57]:
#get input and output
x = df.drop("cnt",axis=1)#.drop("dteday",axis=1)#.drop("start station",axis=1) #.drop("starttime",axis=1)
y = df["cnt"]

In [58]:
#split data into train and test
x_train, x_test ,y_train , y_test = train_test_split(x,y,test_size=0.2,shuffle=False)

In [60]:
#normalise data
ct = make_column_transformer(
 (MinMaxScaler(),["season",
                  "temp","weathersit","workingday","weekday","holiday","yr","hr","mnth",
                 "atemp","hum","windspeed"]))

 

ct.fit(x_train)

x_train_normalized = ct.transform(x_train)

x_test_normalized = ct.transform(x_test)

x_train_normalized.shape

print(x_test_normalized.shape)

(3476, 12)


In [61]:
#create Model structure
with tf.device('/device:CPU:0'):
    model = Sequential()
    model.add(Dense(12, kernel_initializer='normal',  activation='relu', input_shape=(12,) ))    
    model.add(Dense(6, kernel_initializer='normal',activation='relu'))   
    model.add(Dense(6, kernel_initializer='normal',activation='relu'))   
    model.add(Dense(1, kernel_initializer='normal',activation=None))

In [62]:
#set model loss and optimizer
with tf.device('/device:CPU:0'):
    model.compile(optimizer= Adam(learning_rate=0.00001),loss='mae', metrics=['mae','mape'])

In [63]:
#checkpoint to save best Model which isnt overfitted
checkpoint = ModelCheckpoint('Model1', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='min' )

In [65]:
#train Model
with tf.device('/device:CPU:0'):
    history = model.fit(x_train_normalized, y_train.values, epochs=100 , batch_size=10,shuffle=False   )

Epoch 1/100
1391/1391 [==============================] - 1s 768us/step - loss: 173.2507 - mae: 173.2507 - mape: 94.1342
Epoch 2/100
1391/1391 [==============================] - 1s 777us/step - loss: 173.2365 - mae: 173.2365 - mape: 94.0972
Epoch 3/100
1391/1391 [==============================] - 1s 799us/step - loss: 173.2246 - mae: 173.2246 - mape: 94.0603
Epoch 4/100
1391/1391 [==============================] - 1s 780us/step - loss: 173.2100 - mae: 173.2100 - mape: 94.0234
Epoch 5/100
1391/1391 [==============================] - 1s 794us/step - loss: 173.1970 - mae: 173.1970 - mape: 93.9865
Epoch 6/100
1391/1391 [==============================] - 1s 793us/step - loss: 173.1817 - mae: 173.1817 - mape: 93.9495
Epoch 7/100
1391/1391 [==============================] - 1s 749us/step - loss: 173.1679 - mae: 173.1679 - mape: 93.9126
Epoch 8/100
1391/1391 [==============================] - 1s 759us/step - loss: 173.1543 - mae: 173.1543 - mape: 93.8757
Epoch 9/100
1391/1391 [=================

1391/1391 [==============================] - 1s 781us/step - loss: 172.3527 - mae: 172.3527 - mape: 92.1132
Epoch 69/100
1391/1391 [==============================] - 1s 818us/step - loss: 172.3390 - mae: 172.3390 - mape: 92.0957
Epoch 70/100
1391/1391 [==============================] - 1s 799us/step - loss: 172.3264 - mae: 172.3264 - mape: 92.0780
Epoch 71/100
1391/1391 [==============================] - 1s 817us/step - loss: 172.3118 - mae: 172.3118 - mape: 92.0604
Epoch 72/100
1391/1391 [==============================] - 1s 775us/step - loss: 172.2994 - mae: 172.2994 - mape: 92.0428 0s - loss: 159.3092 - mae: 159.3092 - mape: 91.6
Epoch 73/100
1391/1391 [==============================] - 1s 796us/step - loss: 172.2851 - mae: 172.2851 - mape: 92.0252
Epoch 74/100
1391/1391 [==============================] - 1s 748us/step - loss: 172.2718 - mae: 172.2718 - mape: 92.0075
Epoch 75/100
1391/1391 [==============================] - 1s 738us/step - loss: 172.2602 - mae: 172.2602 - mape: 91.9